create table forest_area(
    country_code text,
    country_name text,
    year smallint,
    forest_area_sqkm real
)

create table land_area(
    country_code text,
    country_name text,
    year smallint,
    total_area_mi real
)

create table regions(
    country_name text,
    country_code text,
    region text,
    income_group text
)

**Steps to complete:**

In [ ]:
-- Create a View called “forestation” by joining all three tables - 
-- forest_area, land_area and regions in the workspace.

CREATE VIEW forestation AS
    SELECT fa.country_code
    , fa.country_name
    , fa.year
    , fa.forest_area_sqkm
    , la.total_area_mi
    , (fa.forest_area_sqkm/la.total_area_mi) * (100/2.59) as pct_forest
    , r.region
    , r.income_group
    FROM forest_area fa
    JOIN land_area la
    ON fa.country_code = la.country_code AND
        fa.year = la.year
    JOIN regions r
    ON r.country_code = fa.country_code
    ;


**1\. Global situation**

In [ ]:
-- What was the total forest area (in sq km) of the world in 1990?
SELECT forest_area_sqkm
FROM forestation
WHERE country_name = 'World' and year = '1990'


In [ ]:
-- What was the total forest area (in sq km) of the world in 2016?
SELECT forest_area_sqkm
FROM forestation
WHERE country_name = 'World' AND year = '2016'

In [ ]:
-- c. What was the change (in sq km) in the forest area of the world from 1990 to 2016?

SELECT year
, forest_area_sqkm
, lag(forest_area_sqkm) over (order by year) as previous_area
, forest_area_sqkm - lag(forest_area_sqkm) over (order by year) as change_sqkm
FROM forestation
WHERE country_name = 'World' AND 
    (year = '2016' or year = '1990')

In [ ]:
-- Alternate

With forest_cover AS
(
    SELECT year
    , forest_area_sqkm
    FROM forestation
    WHERE country_name = 'World'
    AND year in (1990, 2016) 
) 
SELECT f1.forest_area_sqkm - f2.forest_area_sqkm AS change_sqkm
FROM forest_cover f1 
JOIN forest_cover f2
ON f1.year > f2.year;

In [ ]:
-- d. What was the percent change in forest area of the world between 1990 and 2016?

WITH forest_cover AS 
(
    SELECT year
    , forest_area_sqkm
    , lag(forest_area_sqkm) OVER (ORDER BY year) as previous_area
    FROM forestation
    WHERE country_name = 'World' AND (year = 2016 or year = 1990)
) 
SELECT *
, forest_area_sqkm - previous_area as coverage_diff
, (forest_area_sqkm - previous_area)/previous_area * 100 as pct_coverage_diff
FROM forest_cover;

In [ ]:
-- Alternate

With forest_cover AS
(
    SELECT year
    , forest_area_sqkm
    FROM forestation
    WHERE country_name = 'World'
    AND year in (1990, 2016) 
) 
SELECT f1.forest_area_sqkm - f2.forest_area_sqkm AS change_sqkm
, (f1.forest_area_sqkm - f2.forest_area_sqkm)/f2.forest_area_sqkm * 100 AS pct_coverage_diff
FROM forest_cover f1 
JOIN forest_cover f2
ON f1.year > f2.year;

In [ ]:
-- e. If you compare the amount of forest area lost between 1990 and 2016, 
-- to which country's total area in 2016 is it closest to?

WITH forest_cover AS 
(
    SELECT year
    , forest_area_sqkm
    , lag(forest_area_sqkm) over (order by year) as previous_area
    FROM forestation
    WHERE country_name = 'World' AND (year = 2016 or year = 1990)
) 
SELECT country_name
, total_area_mi * 2.59 as total_area_sqkm
from forestation
where year = 2016 AND total_area_mi * 2.59 < (SELECT abs(forest_area_sqkm - previous_area) 
                                                FROM forest_cover where year = 2016)
order by total_area_mi * 2.59 desc
limit 1;

In [ ]:
--a. What was the percent forest of the entire world in 2016? 
--b. Which region had the HIGHEST percent forest in 2016, 
-- and which had the LOWEST, to 2 decimal places?
SELECT pct_forest
FROM forestation
WHERE country_name = 'World' and year = 2016;

In [ ]:
-- Highest
SELECT country_name
FROM forestation
WHERE year = 2016 
AND ROUND(pct_forest::NUMERIC , 2) = (SELECT ROUND(MAX(pct_forest)::NUMERIC , 2) 
                                        FROM forestation
                                        WHERE year = 2016);

In [ ]:
-- Lowest
SELECT country_name
FROM forestation
WHERE year = 2016 
AND ROUND(pct_forest::NUMERIC , 2) = (SELECT ROUND(MIN(pct_forest)::NUMERIC , 2) 
                                        FROM forestation
                                        WHERE year = 2016);

**2\. Regional Outlook**

In [ ]:
-- a. What was the percent forest of the entire world in 2016? 
-- Which region had the HIGHEST percent forest in 2016, and 
-- which had the LOWEST, to 2 decimal places?

SELECT country_name
, pct_forest
FROM forestation
WHERE country_name = 'World' AND
    year = 2016;


In [ ]:

With region_forest AS
(
    SELECT region
    , SUM(forest_area_sqkm)/SUM(total_area_mi * 2.59) * 100 as pct_forest
    FROM forestation
    WHERE year = 2016
    GROUP BY region
) 
SELECT * FROM region_forest

In [ ]:
With region_forest AS
(
    SELECT region
    , SUM(forest_area_sqkm)/SUM(total_area_mi * 2.59) * 100 as pct_forest
    FROM forestation
    WHERE year = 2016
    GROUP BY region
) 
SELECT region
, RANK() OVER (ORDER BY pct_forest DESC)
from region_forest
limit 1;

In [ ]:
-- Lowest
With region_forest AS
(
    SELECT region
    , SUM(forest_area_sqkm)/SUM(total_area_mi * 2.59) * 100 as pct_forest
    FROM forestation
    WHERE year = 1990
    GROUP BY region
) 
SELECT region
, RANK() OVER (ORDER BY pct_forest)
from region_forest
limit 1;


In [ ]:
-- c. Based on the table you created, which regions of the world DECREASED in forest area from 1990 to 2016?
With region_forest AS
(
    SELECT region
    , year
    , SUM(forest_area_sqkm)/SUM(total_area_mi * 2.59) * 100 as pct_forest
    FROM forestation
    WHERE year = 1990 OR year = 2016
    GROUP BY region, year
    ORDER BY region, year
),
region_forest_diff AS
( 
    SELECT region
    , year
    , pct_forest
    , lag(pct_forest) OVER (PARTITION BY region ORDER BY year) as previous_pct
    FROM region_forest
) 
SELECT region
, (pct_forest - previous_pct) as pct_forest_diff
FROM region_forest_diff
WHERE (pct_forest - previous_pct) < 0;

In [ ]:
-- Alternate

With region_forest AS
(
    SELECT region
    , year
    , SUM(forest_area_sqkm)/SUM(total_area_mi * 2.59) * 100 as pct_forest
    FROM forestation
    WHERE year = 1990 OR year = 2016
    GROUP BY region, year
    ORDER BY region, year
) 
SELECT r1.region
, (r1.pct_forest - r2.pct_forest) as pct_forest_diff
FROM region_forest r1
JOIN region_forest r2
ON r1.year > r2.year AND r1.region = r2.region
WHERE (r1.pct_forest - r2.pct_forest) < 0;


**<span style="color: #d33682;">3</span>. Country<span style="color: #859900;">-level</span> Detail**

In [ ]:
-- a. Which 5 countries saw the largest amount decrease in forest area from 1990 to 2016? 
-- What was the difference in forest area for each?

WITH forest_cover AS
(
    SELECT country_name
    , year
    , forest_area_sqkm
    , lag(forest_area_sqkm) OVER (PARTITION BY country_name ORDER BY year) as previous_forest_area
    FROM forestation
    WHERE year = 1990 or year = 2016
    ORDER BY country_name, year
),
forest_diff AS
(
    SELECT *
    , forest_area_sqkm - previous_forest_area as diff_forest_area
    , ((forest_area_sqkm - previous_forest_area)/previous_forest_area) * 100 as pct_decrease
    FROM forest_cover 
    WHERE year = 2016 and country_name != 'World'
)
SELECT country_name, diff_forest_area 
FROM forest_diff
ORDER BY diff_forest_area
limit 5;


In [ ]:
WITH forest_cover AS
(
    SELECT country_name
    , year
    , forest_area_sqkm
    , lag(forest_area_sqkm) OVER (PARTITION BY country_name ORDER BY year) as previous_forest_area
    FROM forestation
    WHERE year in (1990, 2016) AND country_name != 'World'
    ORDER BY country_name, year
) 
SELECT f1.country_name
, f1.forest_area_sqkm - f2.forest_area_sqkm AS diff_forest_area
FROM forest_cover f1
JOIN forest_cover f2
ON f1.year > f2.year AND f1.country_name = f2.country_name
ORDER BY f1.forest_area_sqkm - f2.forest_area_sqkm
limit 5;

In [ ]:
-- b. Which 5 countries saw the largest percent decrease in forest area from 1990 to 2016? 
-- What was the percent change to 2 decimal places for each?

WITH forest_cover AS
(
    SELECT country_name
    , year
    , forest_area_sqkm
    , lag(forest_area_sqkm) OVER (PARTITION BY country_name ORDER BY year) as previous_forest_area
    FROM forestation
    WHERE year = 1990 or year = 2016
    ORDER BY country_name, year
),
forest_diff AS
(
    SELECT *
    , forest_area_sqkm - previous_forest_area as diff_forest_area
    , ((forest_area_sqkm - previous_forest_area)/previous_forest_area) * 100 as pct_change
    FROM forest_cover 
    WHERE year = 2016 and country_name != 'World'
)
SELECT country_name
, ROUND(pct_change::NUMERIC, 2) as pct_change 
FROM forest_diff
ORDER BY pct_change
limit 5;


In [ ]:
-- Alternate

WITH forest_cover AS
(
    SELECT country_name
    , year
    , forest_area_sqkm
    , lag(forest_area_sqkm) OVER (PARTITION BY country_name ORDER BY year) as previous_forest_area
    FROM forestation
    WHERE year in (1990, 2016) AND country_name != 'World'
    ORDER BY country_name, year
) 
SELECT f1.country_name
, ROUND(((f1.forest_area_sqkm - f2.forest_area_sqkm)/f2.forest_area_sqkm * 100)::NUMERIC, 2) AS pct_change
FROM forest_cover f1
JOIN forest_cover f2
ON f1.year > f2.year AND f1.country_name = f2.country_name
ORDER BY 2
limit 5;

In [ ]:
--c. If countries were grouped by percent forestation in quartiles, 
--which group had the most countries in it in 2016?

WITH forest_quartile AS
(
    SELECT country_name 
    , CASE 
        WHEN pct_forest <= 25 THEN '<= 25%'
        WHEN pct_forest > 25 AND pct_forest <= 50 THEN '> 25% AND <= 50%'
        WHEN pct_forest > 50 AND pct_forest <= 75 THEN '> 50% AND <= 75%'
        ELSE '> 75%'
    END AS quartile_grp
    FROM forestation
    WHERE year = 2016 AND pct_forest IS NOT NULL
) 
SELECT quartile_grp, COUNT(*) 
FROM forest_quartile
GROUP BY quartile_grp
ORDER BY COUNT(*) DESC;

In [ ]:
--d. List all of the countries that were in the 4th quartile (percent forest > 75%) in 2016

WITH forest_quartile AS
(
    SELECT country_name 
    , CASE 
        WHEN pct_forest <= 25 THEN '<= 25%'
        WHEN pct_forest > 25 AND pct_forest <= 50 THEN '> 25% AND <= 50%'
        WHEN pct_forest > 50 AND pct_forest <= 75 THEN '> 50% AND <= 75%'
        ELSE '> 75%'
    END AS quartile_grp
    FROM forestation
    WHERE year = 2016 AND pct_forest IS NOT NULL
) 
SELECT country_name
FROM forest_quartile
WHERE quartile_grp = '> 75%';

In [ ]:
--e. How many countries had a percent forestation higher than the United States in 2016?

SELECT count(*)
FROM forestation
WHERE year = 2016 
    AND pct_forest IS NOT NULL
    AND pct_forest > (
            SELECT pct_forest
            FROM forestation
            WHERE country_name = 'United States' 
                AND year = 2016
    )